In [46]:
import redis
import faker as fk
import pandas
import random
fake = fk.Faker()

# main client
r = redis.Redis(host='localhost', port=6380, decode_responses=True)

In [47]:
def make_rand_computer():
    os = ['windows', 'macOS', 'linux']
    return {"ip": fake.ipv4(), "name": fake.user_name(), "os": random.choice(os)}

def make_rand_mac():
    return fake.mac_address()

def unravel_dict(dictionary):
    return " ".join([f"{key} {value}"for key,value in dictionary.items()])

def add_computer(computer):
    mac = make_rand_mac()
    r.execute_command(f"HSET mac:{mac} {unravel_dict(computer)}")
    r.execute_command(f"HSET index.mac.by.ip {computer['ip']} {mac}")

    macs = r.execute_command("HGETALL index.mac.by.ip")
    print(macs)
    macs = macs[1::2]
    macs.remove(mac)
    for _ in range(4):
        if macs == []:
            break
        connected = random.choice(macs)
        r.execute_command(f"SADD connections:{mac} {connected}")
        r.execute_command(f"SADD connections:{connected} {mac}")

def initialize_content(n):
    for _ in range(n):
        add_computer(make_rand_computer())


In [48]:
def show_all_macs():
    for key in r.scan_iter("mac:*"):
        print(key)

def show_all_computers():
    for key in r.scan_iter("mac:*"):
        print(key)
        print(r.execute_command(f"HGETALL {key}"))

def show_all_connections():
    for key in r.scan_iter("mac:*"):
        mac = key[len("mac:"):]
        print(mac)
        print(r.execute_command(f"SMEMBERS connections:{mac}"))

def change_ip(old_ip, new_ip):
    r.execute_command(f"WATCH index.mac.by.ip")
    mac_address = r.execute_command(f"HGET index.mac.by.ip {old_ip}")
    r.execute_command(f"MULTI")
    r.execute_command(f"HSET index.mac.by.ip {new_ip} {mac_address}")
    r.execute_command(f"HDEL index.mac.by.ip {old_ip}")
    r.execute_command(f"HSET mac:{mac_address} ip {new_ip}")
    r.execute_command('PUBLISH', 'computer.logs', f"IP of computer {mac_address} has changed from {old_ip} to {new_ip}")
    r.execute_command(f"EXEC")

In [49]:
initialize_content(1)

['153.180.23.139', 'ea:a2:6a:4f:53:16']


In [50]:
show_all_computers()

mac:ea:a2:6a:4f:53:16
['ip', '153.180.23.139', 'name', 'williamjones', 'os', 'linux']


In [51]:
change_ip("119.22.49.29","119.22.49.00")

In [52]:
r.flushdb()

True